In [ ]:
# Dependencies and setup
import requests
import matplotlib.pyplot
import pandas as pd
import numpy as np
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime

# Determine city from latitued and longitude
from citipy import citipy

# Get api key from config file, an untracked file in gitignore
from config import api_key

output_files = 'Output/cities.csv'

# Define range of latitudes and longitudes
lat_r=(-45, 45)
lng_r=(-180, 180)

##### Generate List of Cities

In [ ]:
# Variable with an empty list to hold cities and lat_long
cities = []
lat_long = []

# Randomize creation of latitude & longitude. Set np size to 1000; change if needed
lat = np.random.uniform(low=-45.00, high=45.00, size=1000)
lng = np.random.uniform(low=-180, high = 180, size=1000)
lat_long = zip(lat, lng)

# Find nearest city with geo coordinates
for lat_lng in lat_long:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    if city not in cities:
        cities.append(city)
# Validate sufficient number of cities generated        
len(cities)

##### API Calls

In [ ]:
base_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# URL query
query_url = f'{base_url}appid={api_key}&units={units}&q='

# Print response
response = requests.get(query_url).json()
print(response)

# Dictionary of variables where data from api calls will be stored
city_data = {'city_n':[], 'city_lat':[], 'city_lng':[], 'temp':[], 'humdty':[],
             'cloudns':[], 'wndspd':[], 'cntry':[], 'date':[]}

In [ ]:
city_ttls = len(cities)
r=1

print('Retrieving weather data')
print('-'*26)
for city in cities:
    try:
        print(f"Retrieving {city} -- number {r} of {city_ttls}....")
        r+1
        
#     query_url = base_url + city
#     data = requests.get(query_url).json()